In [1]:
import torch
from torch import nn 
from torch.nn import functional as F
from d2l import torch as d2l
import numpy as np
import pandas as pd
# 参考 https://www.kaggle.com/code/nattoli/simple-resnet-resnext-baseline/notebook

In [3]:
# 使用命令行工具下载数据集：
# https://github.com/Kaggle/kaggle-api
# 还需要配置OpenAPI KEY
# https://www.kaggle.com/{user}/account
# !kaggle competitions download -c classify-leaves

In [2]:
data_file = '../data/ch07-30-classify-leaves'
train_file = f'{data_file}/train.csv'
test_file = f'{data_file}/test.csv'

### 查看训练集label

In [43]:
train_label_file = pd.read_csv(train_file)
train_label_file.describe()

,image,label
count,18353,18353
unique,18353,176
top,images/1081.jpg,maclura_pomifera
freq,1,353


### 处理索引和标签

In [54]:
# 对Label去重排序
leave_labels = sorted(list(set(train_label_file['label'])))
label_count = len(leave_labels)
print('Class number: ', label_count)
leave_labels[:10]

Class number:  176


['abies_concolor',
 'abies_nordmanniana',
 'acer_campestre',
 'acer_ginnala',
 'acer_griseum',
 'acer_negundo',
 'acer_palmatum',
 'acer_pensylvanicum',
 'acer_platanoides',
 'acer_pseudoplatanus']

In [58]:
class_to_num = dict(zip(leave_labels, range(label_count)))
# 数字映射到类
num_to_class = {v: k for k, v in class_to_num.items()}


In [ ]:
from torch.utils.data import Dataset

class LeavesDataset(Dataset):
    def __init__(self, parent, img_path, data_type='train') -> None:
        super().__init__()

In [39]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

ts = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)
train_path = path.join(data_file, 'torch', 'train')
data_train = datasets.ImageFolder(train_path, transform=ts)
data_loader = DataLoader(data_train, batch_size=64, shuffle=True, num_workers=8)

for i, data in enumerate(data_loader):
    images, labels = data
    # print('data.shape', data.shape)
    print(images.shape)
    print(labels)
    break

torch.Size([64, 3, 224, 224])
tensor([170,  45,   7,  74, 168,  59,  25,  70,  79, 130,   6,  12, 168,  31,
         62,  69, 170,  85, 126,  22, 173,  12, 120,  78,  24, 124,   5,  92,
         77,  72,  42, 160,  74,  72,   6, 125, 137,  40,  72,  27, 132,  81,
         95, 149, 111, 173,  99, 175,  12,  33, 133,  47, 162,  57,  78,  32,
         40,  23,  15,  28, 156,  36,  10, 127])


### 实现加载数据的DataSet

In [5]:
from torch.utils.data import Dataset
from PIL import Image

class MyImageDataSet(Dataset):
    def __init__(self, parent, img_paths, transform=None) -> None:
        super().__init__()
        self.parent = parent
        self.img_paths = img_paths
        self.transform = transform
    
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        image = np.array(image)
        if self.transform:
            self.transform(image)
        return image